In [119]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from src.catching import attempt_catch
from src.pokemon import PokemonFactory, StatusEffect

In [120]:
factory = PokemonFactory("pokemon.json")
df = pd.read_json("pokemon.json")
pokemons = list(df.columns)
pokeballs = ["pokeball", "ultraball", "fastball", "heavyball"]

In [121]:
def estimate_catchrate(pokemon_instance, pokeball, noise, n):
    return np.average([attempt_catch(pokemon_instance, pokeball, noise)[0] for _ in range(n)])

### a) Ejecutando la función 100 veces, para cada Pokemon en condiciones ideales (HP:100 %, LVL 100) ¿Cuál es la probabilidad de captura promedio para cada pokebola?

In [122]:
results = {}
for ball in pokeballs:
    ball_rates = {}
    for pokemon in pokemons:
        bicho = factory.create(pokemon, 100, StatusEffect.NONE, 1)
        ball_rates[pokemon] = estimate_catchrate(bicho, ball, 0, 3000)
    # print(f"Ball type: {ball}, Success Rate: {np.average(ball_rates)}")
    results[ball] = ball_rates
dataframe = pd.DataFrame.from_dict(results)
fig = px.bar(x=list(results.keys()), y=list(map(lambda v: np.average(list(v.values())), results.values())), title="Average capture probability by Pokeball")

In [123]:
fig.show()

### 1b) ¿Es cierto que algunas pokebolas son más o menos efectivas dependiendo de propiedades intrinsecas de cada Pokemon? Justificar.
Asumiendo que las variantes de las pokebolas son simplemente variaciones de la pokebola base con diferentes efectividades, efectividad fastball / efectividad pokeball deberia mantenerse igual para todos los pokemons

In [124]:
from itertools import islice
result2 = {}
base_pokeball = results["pokeball"]
for (pokeball, pokemons) in islice(results.items(), 1, None):
    result2[pokeball] = { pokemon: catchrate / base_pokeball[pokemon] for (pokemon, catchrate) in pokemons.items() }

fig2 = px.bar(result2, title="Efectividad comparativa entre tipos de pokebolas", barmode='group')
fig2.show()

Podemos ver desde el grafico que algunos tipos pokebolas son (en comparacion a la pokebola base)
mas efectivas en ciertos pokemons, por ejemplo las fastball en mewtwo, la heavyball en snorlax

### 2a)¿Las condiciones de salud tienen algún efecto sobre la efectividad de la captura? Si es ası́, ¿Cuál es más o menos efectiva?

In [125]:
pokemon_test = df.columns[0]
variants = { status: factory.create(pokemon_test, 50, status, 1) for status in StatusEffect }
catchrate_by_variant = { status: estimate_catchrate(pokemon, pokeballs[0], 1, 3000) for (status, pokemon) in variants.items() }


fig3 = px.bar(x = list(map(lambda status : status.name, catchrate_by_variant.keys())), y = list(catchrate_by_variant.values()), title = "Efectividad entre efectos de estado")
fig3.show()

Podemos ver que los mas efectivos son SLEEP y FREEZE.

### 2b) ¿Cómo afectan los puntos de vida a la efectividad de la captura? Sugerencia: Elegir uno o dos Pokemones y manteniendo el resto de los parámetros constantes, calcular la probabilidad de captura para distintos HP %

In [126]:
import decimal

def drange(x, y, jump):
  while x < y:
    yield float(x)
    x += decimal.Decimal(jump)

health_vars = { percentage: factory.create(pokemon_test, 50, StatusEffect.NONE, percentage) for percentage in drange(0, 1, 0.05) }
catchrate_by_health = { percentage: estimate_catchrate(pokemon, pokeballs[0], 1, 8000) for (percentage, pokemon) in health_vars.items() }

In [127]:
fig4 = px.line(x = catchrate_by_health.keys(), y = catchrate_by_health.values(), title = f"Posibilidad de captura de {pokemon_test} con respecto a su vida")
fig4.update_xaxes(tickformat=".1%")
fig4.update_yaxes(tickformat=".1%")
fig4.show()

Podemos ver que a medida a mayor vida menos posibilidad de captura

### 2c) ¿Qué parámetros son los que más afectan la probabilidad de captura?

Para ver que parámetros afectan más a la probabilidad de captura, calculo la diferencia entre variantes del mismo parámetro. La diferencia entre el mínimo probabilidad de captura y el máximo para cierto pokemón será la respuesta.
Los parametros son:
- La vida
- El Efecto de estado
Debido que la pokebola depende del pokemon, no considero que sea un parámetro tan fácil de modificar.
Tomo un Jolteon como ejemplo.

In [128]:
min_health_catchrate = min(catchrate_by_health.values())
max_health_catchrate = max(catchrate_by_health.values())
catchrate_health_variance = max_health_catchrate - min_health_catchrate

# catchrate_by_variant = { status: estimate_catchrate(pokemon, pokeballs[0], 1, 3000) for (status, pokemon) in variants.items() }

min_status_catchrate = min(catchrate_by_variant.values())
max_status_catchrate = max(catchrate_by_variant.values())
catchrate_status_variance = max_status_catchrate - min_status_catchrate

healthy_pokemon = factory.create(pokemon_test, 50, StatusEffect.NONE, 1)
catchrate_by_pokeball = { pokeball: estimate_catchrate(healthy_pokemon, pokeball, 1, 3000) for pokeball in pokeballs }

min_pokeball_catchrate = min(catchrate_by_pokeball.values())
max_pokeball_catchrate = max(catchrate_by_pokeball.values())
catchrate_pokeball_variance = max_pokeball_catchrate - min_pokeball_catchrate

fig4 = px.bar(x = ["Health", "Status", "Pokeball"], y = [catchrate_health_variance, catchrate_status_variance, catchrate_pokeball_variance], title = "Catch Rate Variance by Health and Status for Jolteon")
fig4.update_xaxes(title_text = "Type of Parameter")
fig4.update_yaxes(title_text = "Variance")
fig4.show()

Para Jolteon en particular, la mayor varianza sucede en el tipo de pokebola que se usa. Debido a que la pokebola depende de los atributos del pokemon, esto puede cambiar.

In [129]:
# Funcion devuelve una tupla (varianza_por_tipo, max y minimo por tipo)
def calculate_all_variances(pokemon, level, iterations):
    health_vars = { percentage: factory.create(pokemon_test, 50, StatusEffect.NONE, percentage) for percentage in drange(0, 1, 0.05) }
    catchrate_by_health = { percentage: estimate_catchrate(pokemon, pokeballs[0], 1, iterations) for (percentage, pokemon) in health_vars.items() }

    variants = { status: factory.create(pokemon, level, status, 1) for status in StatusEffect }
    catchrate_by_variant = { status.name: estimate_catchrate(pokemon, pokeballs[0], 1, iterations) for (status, pokemon) in variants.items() }

    healthy_pokemon = factory.create(pokemon, level, StatusEffect.NONE, 1)
    catchrate_by_pokeball = { pokeball: estimate_catchrate(healthy_pokemon, pokeball, 1, 3000) for pokeball in pokeballs }
    return ({
        "health": max(catchrate_by_health.values()) - min(catchrate_by_health.values()),
        "status": max(catchrate_by_variant.values()) - min(catchrate_by_variant.values()),
        "pokeball": max(catchrate_by_pokeball.values()) - min(catchrate_by_pokeball.values())
    }, {
        "health": {
            "max": max(catchrate_by_health.items(), key = lambda k: k[1]),
            "min": min(catchrate_by_health.items(), key = lambda k: k[1])
        },
        "status": {
            "max": max(catchrate_by_variant.items(), key = lambda k: k[1]),
            "min": min(catchrate_by_variant.items(), key = lambda k: k[1])
        },
        "pokeball": {
            "max": max(catchrate_by_pokeball.items(), key = lambda k: k[1]),
            "min": min(catchrate_by_pokeball.items(), key = lambda k: k[1])
        }
    })

all_pokemons_variances = { pokemon: calculate_all_variances(pokemon, 10, 3000)[0] for pokemon in pokemons }

df = pd.DataFrame.from_dict(all_pokemons_variances)
df = df.assign(mean=df.mean(axis=1))
print(df)
fig5 = px.bar(df, barmode="group")
fig5.show()

           jolteon  caterpie   snorlax      onix    mewtwo      mean
health    0.117667  0.113000  0.126333  0.121000  0.124000  0.120400
status    0.069000  0.226000  0.036333  0.072667  0.005333  0.081867
pokeball  0.218667  0.252333  0.044667  0.059667  0.015667  0.118200


El grafico nos muestra cuales son las varianzas maximas y minimas por cada pokemon. Podemos ver como el estado afecta en particular mucho a caterpie.
Agarrando el promedio de todas las varianzas, parece que la vida y la pokebola afectan bastante en comparacion al estado y nivel.

### 2d) Teniendo en cuenta uno o dos pokemones distintos: ¿Qué combinación de condiciones (propiedades mutables) y pokebola conviene utilizar para capturarlos?

In [130]:
# Como pokemones representativos vamos a usar a caterpie y a snorlax
caterpie = "caterpie"
snorlax = "snorlax"

caterpie_varianzas = calculate_all_variances(caterpie, 10, 3000)
df = pd.DataFrame.from_dict(caterpie_varianzas[1])
print("CATERPIE")
print(df.head())
snorlax_varianzas = calculate_all_variances(snorlax, 10, 3000)
df2 = pd.DataFrame.from_dict(snorlax_varianzas[1])
print("\n\nSNORLAX")
print(df2.head())

CATERPIE
                         health                        status  \
max  (0.0, 0.19133333333333333)  (FREEZE, 0.5936666666666667)   
min                (0.9, 0.075)                 (NONE, 0.347)   

                            pokeball  
max  (ultraball, 0.5836666666666667)  
min  (heavyball, 0.3273333333333333)  


SNORLAX
                          health                         status  \
max   (0.1, 0.19266666666666668)  (FREEZE, 0.07266666666666667)   
min  (0.9500000000000001, 0.068)                  (NONE, 0.043)   

                             pokeball  
max  (heavyball, 0.09733333333333333)  
min  (pokeball, 0.030333333333333334)  


Podemos ver como el mayor porcentaje de captura para caterpie sucede con FREEZE, vida lo menor posible y con una ultraball.
Para snorlax, tambien es FREEZE, vida minima pero con una heavy ball.

### 2e) A partir del punto anterior, ¿serı́a efectiva otra combinación de parámetros teniendo en cuenta un nivel del pokemon más bajo (o más alto)?

In [131]:
# Realizamos el mismo procedimiento con vida mayor
caterpie_varianzas = calculate_all_variances(pokemon = caterpie, level = 100, iterations = 3000)
df = pd.DataFrame.from_dict(caterpie_varianzas[1])
print("CATERPIE")
print(df.head())
snorlax_varianzas = calculate_all_variances(pokemon = snorlax, level = 100, iterations = 3000)
df2 = pd.DataFrame.from_dict(snorlax_varianzas[1])
print("\n\nSNORLAX")
print(df2.head())

CATERPIE
                          health                       status  \
max   (0.1, 0.19366666666666665)  (SLEEP, 0.6016666666666667)   
min  (0.9500000000000001, 0.075)   (NONE, 0.3516666666666667)   

                             pokeball  
max                (ultraball, 0.606)  
min  (heavyball, 0.32666666666666666)  


SNORLAX
                         health                        status  \
max  (0.0, 0.19066666666666668)  (SLEEP, 0.06833333333333333)   
min  (0.9, 0.07233333333333333)  (NONE, 0.029333333333333333)   

                             pokeball  
max  (heavyball, 0.09533333333333334)  
min                 (pokeball, 0.033)  


# Parece que el nivel no afecta que parametros usar, aunque afecte los valores en particular